In [6]:
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm, tqdm_notebook
import os
import time
import glob
import shutil
import os
import random
from PIL import Image
import cv2
import PythonMagick

In [9]:
df = pd.read_json('data/oracle-cards-20200531050621.json')
df.head()

,all_parts,arena_id,artist,artist_ids,booster,border_color,card_back_id,card_faces,cmc,collector_number,...,set_type,set_uri,story_spotlight,tcgplayer_id,textless,toughness,type_line,uri,variation,watermark
0,NaN,NaN,Terese Nielsen,[eb55171c-2342-45f4-a503-2d5a75baf752],True,white,0aeebaf5-8c7d-4636-9e82-8c27447861f7,NaN,3.0,319,...,core,https://api.scryfall.com/sets/230f38aa-9511-4d...,False,3094.0,False,NaN,Artifact,https://api.scryfall.com/cards/86bf43b1-8d4e-4...,False,NaN
1,NaN,NaN,Steven Belledin,[f07d73b9-52a0-4fe5-858b-61f7b42174a5],True,black,0aeebaf5-8c7d-4636-9e82-8c27447861f7,NaN,1.0,71,...,core,https://api.scryfall.com/sets/e03ee1c0-ecd2-4f...,False,69965.0,False,NaN,Enchantment â€” Aura,https://api.scryfall.com/cards/7050735c-b232-4...,False,NaN
2,NaN,NaN,Jonas De Ro,[561ebf9e-8d93-4b57-8156-8826d0c19601],False,black,0aeebaf5-8c7d-4636-9e82-8c27447861f7,NaN,2.0,34,...,commander,https://api.scryfall.com/sets/0fa3ccbb-d86d-4a...,False,196536.0,False,NaN,Sorcery,https://api.scryfall.com/cards/e718b21b-46d1-4...,False,NaN
3,NaN,NaN,John Matson,[a1685587-4b55-446b-b420-c37b202ed3f1],True,white,0aeebaf5-8c7d-4636-9e82-8c27447861f7,NaN,2.0,100,...,core,https://api.scryfall.com/sets/e70c8572-4732-4e...,False,12835.0,False,2,Creature â€” Bird,https://api.scryfall.com/cards/036ef8c9-72ac-4...,False,NaN
4,NaN,NaN,Ron Spencer,[dab52c11-0564-4207-a4a1-c1735c946a65],True,black,0aeebaf5-8c7d-4636-9e82-8c27447861f7,NaN,2.0,47,...,expansion,https://api.scryfall.com/sets/78ced61b-4b8a-4b...,False,6412.0,False,1,Creature â€” Sponge,https://api.scryfall.com/cards/b125d1e7-5d9b-4...,False,NaN


In [10]:
print(df.shape)
#df.drop_duplicates(subset='oracle_id', inplace=True)
#print(df.shape)

(21370, 70)


In [11]:
print(df.columns)

Index(['all_parts', 'arena_id', 'artist', 'artist_ids', 'booster',
       'border_color', 'card_back_id', 'card_faces', 'cmc', 'collector_number',
       'color_identity', 'color_indicator', 'colors', 'digital', 'edhrec_rank',
       'flavor_name', 'flavor_text', 'foil', 'frame', 'frame_effects',
       'full_art', 'games', 'hand_modifier', 'highres_image', 'id',
       'illustration_id', 'image_uris', 'lang', 'layout', 'legalities',
       'life_modifier', 'loyalty', 'mana_cost', 'mtgo_foil_id', 'mtgo_id',
       'multiverse_ids', 'name', 'nonfoil', 'object', 'oracle_id',
       'oracle_text', 'oversized', 'power', 'preview', 'prices',
       'printed_name', 'prints_search_uri', 'promo', 'promo_types', 'rarity',
       'related_uris', 'released_at', 'reprint', 'reserved', 'rulings_uri',
       'scryfall_set_uri', 'scryfall_uri', 'set', 'set_name', 'set_search_uri',
       'set_type', 'set_uri', 'story_spotlight', 'tcgplayer_id', 'textless',
       'toughness', 'type_line', 'uri', 'var

In [12]:
url_list = [jsd for jsd in df['image_uris'].values]
#print(url_list[0])
http_list = []
for url in url_list:
    try:
        http_list.append(url.get('art_crop'))
    except:
        http_list.append(np.nan)
print(len(url_list))
print(len(http_list))
print(http_list[:10])

21370
21370
['https://img.scryfall.com/cards/art_crop/front/8/6/86bf43b1-8d4e-4759-bb2d-0b2e03ba7012.jpg?1562242171', 'https://img.scryfall.com/cards/art_crop/front/7/0/7050735c-b232-47a6-a342-01795bfd0d46.jpg?1562830795', 'https://img.scryfall.com/cards/art_crop/front/e/7/e718b21b-46d1-4844-985c-52745657b1ac.jpg?1568003608', 'https://img.scryfall.com/cards/art_crop/front/0/3/036ef8c9-72ac-46ce-af07-83b79d736538.jpg?1562730661', 'https://img.scryfall.com/cards/art_crop/front/b/1/b125d1e7-5d9b-4997-88b0-71bdfc19c6f2.jpg?1562863790', 'https://img.scryfall.com/cards/art_crop/front/e/0/e0f83824-43c6-4101-88fd-9109958b23e2.jpg?1557576051', 'https://img.scryfall.com/cards/art_crop/front/f/e/feeee859-f64a-4cd8-be0b-ad60cff8812e.jpg?1562537400', 'https://img.scryfall.com/cards/art_crop/front/3/c/3c97d419-6fee-4a32-9e64-8a827da59934.jpg?1559592454', 'https://img.scryfall.com/cards/art_crop/front/2/8/280f2a85-1900-460b-a768-164fc2dea636.jpg?1584830293', 'https://img.scryfall.com/cards/art_crop/f

In [13]:
query_df = df[['color_identity', 'oracle_id']]
query_df['url'] = http_list
query_df.dropna(inplace=True)
query_df['id'] = [str(image_url.split('?')[-1]) for image_url in query_df['url']]
print(query_df.shape)
query_df.tail()

C:\Users\Wolfgang\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Wolfgang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Wolfgang\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/panda

(21177, 4)


,color_identity,oracle_id,url,id
21365,[W],fff6f2d0-22b5-41d0-99dc-bc817c3bd166,https://img.scryfall.com/cards/art_crop/front/...,1562875264
21366,[B],fffa9334-3576-4f70-9605-2ad062cdbc69,https://img.scryfall.com/cards/art_crop/front/...,1562816474
21367,"[R, W]",fffaa634-42a0-4038-b027-24f28754fec7,https://img.scryfall.com/cards/art_crop/front/...,1562253988
21368,"[G, R]",fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91,https://img.scryfall.com/cards/art_crop/front/...,1562788629
21369,[W],ffff90c3-63c4-4dee-a21d-6b2b113f4f80,https://img.scryfall.com/cards/art_crop/front/...,1562573847


In [14]:
print(query_df['url'].iloc[2000])

https://img.scryfall.com/cards/art_crop/front/c/f/cf7f2954-65f6-4944-ae68-8992cb0268f4.jpg?1562936706


In [15]:
print(query_df['url'].loc[query_df['id'] == '1551144270'])

Series([], Name: url, dtype: object)


In [16]:
query_df['filename'] = ['{}_{}_{}.jpg'.format(str(orid), str(cardid), color) for orid, cardid, color in zip(query_df['oracle_id'], query_df['id'], query_df['color_identity'])]

'''
print(query_df.head())
current_files = os.listdir('images')
print(current_files[:10])
query_df = query_df.loc[query_df['filename'].isin(current_files)]
'''
print(query_df.shape)
query_df.head()

C:\Users\Wolfgang\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(21177, 5)


,color_identity,oracle_id,url,id,filename
0,[],0004ebd0-dfd6-4276-b4a6-de0003e94237,https://img.scryfall.com/cards/art_crop/front/...,1562242171,0004ebd0-dfd6-4276-b4a6-de0003e94237_156224217...
1,[U],0006faf6-7a61-426c-9034-579f2cfcfa83,https://img.scryfall.com/cards/art_crop/front/...,1562830795,0006faf6-7a61-426c-9034-579f2cfcfa83_156283079...
2,[G],0007c283-5b7a-4c00-9ca1-b455c8dff8c3,https://img.scryfall.com/cards/art_crop/front/...,1568003608,0007c283-5b7a-4c00-9ca1-b455c8dff8c3_156800360...
3,[U],000d5588-5a4c-434e-988d-396632ade42c,https://img.scryfall.com/cards/art_crop/front/...,1562730661,000d5588-5a4c-434e-988d-396632ade42c_156273066...
4,[U],000e5d65-96c3-498b-bd01-72b1a1991850,https://img.scryfall.com/cards/art_crop/front/...,1562863790,000e5d65-96c3-498b-bd01-72b1a1991850_156286379...


In [17]:
print(len(list(set(query_df['oracle_id']))))

21177


In [18]:
query_df['counter'] = 1
id_df = query_df['counter'].groupby(query_df['id']).sum().reset_index()
id_df.sort_values('counter', ascending=False,inplace=True)
id_df.head()

C:\Users\Wolfgang\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,id,counter
15308,1562942414,3
18741,1576381808,3
6913,1562629522,2
12185,1562900088,2
13516,1562918634,2


In [19]:
query_df['url'].loc[query_df['id'] == '1530592568']

Series([], Name: url, dtype: object)

In [21]:
dir_location = 'data/images'


pbar = tqdm_notebook(total=len(query_df['url'].values))
for i, image_url in enumerate(query_df['url'].values):
    if i >= 0: # incase we get booted we can restart from last image index
        img_data = requests.get(image_url).content
        filename = image_url.split('?')[-1]
        color = query_df['color_identity'].iloc[i]
        with open(os.path.join(dir_location, query_df['filename'].iloc[i]), 'wb') as handler:
            handler.write(img_data)
        time.sleep(0.1)
    pbar.update()
pbar.close()

In [129]:
## searches for duplicates in the training dataset ##
## using raster hashes~~~~~~ ##

train_dir = 'images'
duplicate_img_dir= 'duplicate_images'

def is_similar(image1, image2):
    return image1.shape == image2.shape and not(np.bitwise_xor(image1,image2).any())

filelist = os.listdir(train_dir)
filelist = [os.path.join(train_dir, f) for f in filelist]

for i, file in enumerate(filelist):
    print('On File {}/{}:'.format(i, len(filelist)))
    this_file = PythonMagick.Image(file).signature()
    pbar = tqdm_notebook(total=len(range(i+1, len(filelist))))
    for j in range(i+1, len(filelist)):
        next_file = PythonMagick.Image(filelist[j]).signature()
        #if is_similar(this_file, next_file):
        if this_file == next_file:
            print('Similar file found: {}'.format(filelist[j]))
            try:
                os.rename(filelist[j], os.path.join(duplicate_img_dir, os.path.basename(filelist[j])))
                print('Removal success: {}'.format(filelist[j]))
            except WindowsError:
                print('Removal Failure: {}'.format(filelist[j]))
                try:
                    os.remove(filelist[j])
                    print('Deletion success: {}'.format(filelist[j]))
                except WindowsError:
                    print('Error deleting {}'.format(filelist[j]))
            j -= 1
        pbar.update()
    pbar.close()













  1%|          | 2212/186898 [02:01<2:30:49, 20.41it/s]

On File 0/186899:


KeyboardInterrupt: 

In [3]:
import numpy as np
a = np.random.normal(size=(4, 42, 1024))
a = np.transpose(a, (0,2,1))
b = np.random.normal(size=(42, 1024))
r = np.tensordot(a, b, [3, 0])
print(r.shape)

IndexError: tuple index out of range